<a href="https://colab.research.google.com/github/suryap175/Orange-Fruit-Prediction/blob/main/PredBinarySem6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential 

Load and Read From Google Drive


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1.0 / 255, 
        rotation_range=30,  
        zoom_range = 0.15,
        shear_range = 0.2,    
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory( r'/content/drive/MyDrive/New Orange Images Dataset/Orange training',
                                                  target_size=(64,64),
                                                  batch_size=32,
                                                  class_mode='binary')

val_set = val_datagen.flow_from_directory( r'/content/drive/MyDrive/New Orange Images Dataset/orange testing',
                                          target_size=(64,64),
                                          batch_size=32,
                                          class_mode='binary')

In [ ]:
cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#------------------------------ VGG19 ----------------------------------------------------
Base_model = tf.keras.applications.VGG19(input_shape=[64, 64, 3],
                                              include_top=False,
                                              weights='imagenet')
Base_model.trainable = True
model = tf.keras.Sequential([Base_model,
                             keras.layers.GlobalAveragePooling2D(),
                             keras.layers.Dense(64, activation='relu'),
                             keras.layers.Dense(1, activation='sigmoid')])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

#------------------------------ MobileNetV2 ----------------------------------------------------
Base_model = tf.keras.applications.MobileNetV2(input_shape=[64, 64, 3],
                                              include_top=False,
                                              weights='imagenet')
Base_model.trainable = False
model1 = tf.keras.Sequential([Base_model,
                             keras.layers.GlobalAveragePooling2D(),
                             keras.layers.Dense(64, activation='relu'),
                             keras.layers.Dense(1, activation='sigmoid')])

model1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
#------------------------------ LeNet ----------------------------------------------------
cnn2 = tf.keras.models.Sequential()

cnn2.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="tanh", input_shape=[64, 64, 3]))
cnn2.add(tf.keras.layers.BatchNormalization())
cnn2.add(tf.keras.layers.AvgPool2D(pool_size=2, strides=2, padding='valid'))

cnn2.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="tanh"))
cnn2.add(tf.keras.layers.AvgPool2D(pool_size=2, strides=2, padding='valid'))

cnn2.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="tanh"))

cnn2.add(tf.keras.layers.Flatten())

cnn2.add(tf.keras.layers.Dense(units=64, activation='relu'))
cnn2.add(tf.keras.layers.Dense(units=32, activation='relu'))

cnn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

cnn2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:

print("--------------------------------------------VGG---------------------------------------------------")
vgg = model.fit(
          training_set,
          steps_per_epoch=15,
          epochs=25,
          validation_data = val_set,
          validation_steps=10
         )
print("-------------------------------------------Mobile--------------------------------------------------")
mobile = model1.fit(
          training_set,
          steps_per_epoch=15,
          epochs=25, 
          validation_data = val_set,
          validation_steps=10
         )
print("---------------------------------------------CNN---------------------------------------------------")
history = cnn.fit(
          training_set,
          steps_per_epoch=15,
          epochs=25, 
          validation_data = val_set,
          validation_steps=10
        )
print("---------------------------------------------LeNet-----------------------------------------------")
history2 = cnn2.fit(
          training_set,
          steps_per_epoch=15,
          epochs=25, 
          validation_data = val_set,
          validation_steps=10
        )


In [ ]:
epochs = 25
acc = vgg.history['accuracy']
val_acc = vgg.history['val_accuracy']
loss = vgg.history['loss']
val_loss = vgg.history['val_loss']
epochs_range = range(epochs)
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Testing Accuracy')
plt.legend(loc='lower right')
plt.title('vgg Working Accuracay')
plt.show()
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('vgg Training and Validation Loss')
plt.show()


from matplotlib import pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs_range = range(epochs)
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Testing Accuracy')
plt.legend(loc='lower right')
plt.title(' cnn Working Accuracay')
plt.show()
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('cnn Training and Validation Loss')
plt.show()


from matplotlib import pyplot as plt
acc = mobile.history['accuracy']
val_acc = mobile.history['val_accuracy']
loss=mobile.history['loss']
val_loss = mobile.history['val_loss']
epochs_range = range(epochs)
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Testing Accuracy')
plt.legend(loc='lower right')
plt.title(' MobileNet Working Accuracay')
plt.show()
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('MobileNetv2 Training and Validation Loss')
plt.show()

from matplotlib import pyplot as plt
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']
loss=history2.history['loss']
val_loss=history2.history['val_loss']
epochs_range = range(epochs)
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Testing Accuracy')
plt.legend(loc='lower right')
plt.title(' LeNet Working Accuracay')
plt.show()
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('LeNet Training and Validation Loss')
plt.show()


In [ ]:
import numpy as np
from keras.preprocessing import image
p=r"/content/drive/MyDrive/New Orange Images Dataset/orange testing/Oranges/3cbfec1259635898efae5b57ea3ddea3.jpg"
test_image = image.load_img(p, target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image/=255.0
result = cnn.predict(test_image)
print("---- By CNN ----")
if result<0.5:
  print('Selected Fruit is Orange') 
else:
  print('Selected Fruit is Other than Orange')

result_vgg = model.predict(test_image)
print("---- By VGG ----")
if result_vgg<0.5:
  print('Selected Fruit is Orange') 
else:
  print('Selected Fruit is Other than Orange')

result_mobile = model1.predict(test_image)
print("---- By MobileNet ----")
if result_mobile<0.5:
  print('Selected Fruit is Orange') 
else:
  print('Selected Fruit is Other than Orange')

result_lenet = cnn2.predict(test_image)
print("---- By LeNet ----")
if result_lenet<0.5:
  print('Selected Fruit is Orange') 
else:
  print('Selected Fruit is Other than Orange')


In [ ]:
def show(pred,img):

  if pred<0.5: str = 'Selected Fruit is Orange'
  else: str = 'Selected Fruit is Other than Orange'

  plt.imshow(img)
  plt.axis('on')
  plt.title(str)
  plt.show()

img = image.load_img(p)
print("------ CNN ------")
show(result,img)
print("------ VGG-19 ------")
show(result_vgg,img)
print("------ MobileNet_V2 ------")
show(result_mobile,img)
print("------ LeNet ------")
show(result_lenet,img)


In [ ]:
'''#----------------------------------------Save Weight------------------------------------
m_json = cnn.to_json()  
with open("/content/drive/MyDrive/cnn.json", "w") as json_file:  
    json_file.write(m_json)  
cnn.save_weights("/content/drive/MyDrive/Orange_Fruit_Weights_cnn.h5")

#-----------------------vgg-----------------
m_json = model.to_json()  
with open("/content/drive/MyDrive/vgg.json", "w") as json_file:  
    json_file.write(m_json)  
model.save_weights("/content/drive/MyDrive/Orange_Fruit_Weights_vgg.h5")
#-----------------------mobilenet----------
m_json = model1.to_json()  
with open("/content/drive/MyDrive/mobile.json", "w") as json_file:  
    json_file.write(m_json)  
model1.save_weights("/content/drive/MyDrive/Orange_Fruit_Weights_mobile.h5")
#-----------------------Lenet--------------
m_json = cnn2.to_json()  
with open("/content/drive/MyDrive/lenet.json", "w") as json_file:  
    json_file.write(m_json)  
cnn2.save_weights("/content/drive/MyDrive/Orange_Fruit_Weights_lenet.h5")'''